Todays Lab: CRUD operations(Cosmosdb)
 
Make sure you create an cosmosdb resource in Azure just like i did in the lecture using docker and powershell. Or simply do it through the portal manually. I have showed you both. And after that do the following: 
 
Create a student database:
1. Connect to the database using the MongoDB API.
2. Insert documents — e.g., student names and grades.
3. Query documents — find all students with grade above 80.
4. Update a student record (e.g., change grade).
5. Delete a student record.


Here are some usefull links to check out:

https://learn.microsoft.com/en-us/azure/developer/azure-developer-cli/install-azd?tabs=winget-windo…
https://docs.docker.com/desktop/setup/install/windows-install/
https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/quickstart-python

In [31]:
from azure.identity import DefaultAzureCredential
from pymongo import MongoClient
from pymongo.auth_oidc import OIDCCallback, OIDCCallbackContext, OIDCCallbackResult


In [32]:
# 1, 定义一个类，该 AzureIdentityTokenCallback 类定义一个 fetch 函数，该类采用 OIDCCallbackContext 参数并返回一个 OIDCCallbackResult。


class AzureIdentityTokenCallback(OIDCCallback):
    def __init__(self, credential):
        self.credential = credential

    def fetch(self, context: OIDCCallbackContext) -> OIDCCallbackResult:
        token = self.credential.get_token(
            "https://ossrdbms-aad.database.windows.net/.default").token
        return OIDCCallbackResult(access_token=token)

In [33]:
# 2, 使用自定义处理程序类并传入 DefaultAzureCredential 类型的新实例

credential = DefaultAzureCredential()

authProperties = {"OIDC_CALLBACK": AzureIdentityTokenCallback(credential)}

In [34]:
# # 3,使用您的群集名称和适用于 MongoDB vCore 的 Azure Cosmos DB 已知最佳实践配置选项来构建 MongoClient 实例。 此外，请配置自定义身份验证机制。

# clusterName = "<azure-cosmos-db-mongodb-vcore-cluster-name>"

# client = MongoClient(
#     f"mongodb+srv://{clusterName}.global.mongocluster.cosmos.azure.com/",
#     connectTimeoutMS=120000,
#     tls=True,
#     retryWrites=True,
#     authMechanism="MONGODB-OIDC",
#     authMechanismProperties=authProperties
# )

# CONNECTION_STRING = os.environ.get("COSMOS_CONNECTION_STRING")
import os
from dotenv import load_dotenv

load_dotenv(override=True)
CONNECTION_STRING = os.getenv("CONNECTION_STRING")
client = MongoClient(CONNECTION_STRING)


# for prop, value in vars(client.options).items():
#     print("Property: {}: Value: {} ".format(prop, value))

database = client.get_database("student")
collection = database.get_collection("my_student")


/tmp/ipykernel_61589/616891190.py:20: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = MongoClient(CONNECTION_STRING)
/opt/conda/envs/machine_learning/lib/python3.13/site-packages/pymongo/synchronous/mongo_client.py:1758: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self._resolve_srv()


In [35]:
filter = {
    "name": "Linda"
}
existing_document = collection.find_one(filter)
print(f"Read document:\t{existing_document}")

Read document:	{'_id': ObjectId('68ecc83cf7a83f60490b9441'), 'name': 'Linda', 'student_id': '01', 'grade': 98}


In [36]:
# Upsert documents

new_document = {
        "name": "John",
        "student_id": "07",
        "grade": 78
}

filter = {
    "name": "John",
}

payload = {
    "$set": new_document
}
result = collection.update_one(filter, payload, upsert=True)


In [37]:
# Query documents

filter = {
    "grade": {"$gt": 80}
}
matched_documents = collection.find(filter)

for document in matched_documents:
    print(f"Found document:\t{document}")

Found document:	{'_id': ObjectId('68ecc83cf7a83f60490b9441'), 'name': 'Linda', 'student_id': '01', 'grade': 98}
Found document:	{'_id': ObjectId('68ecc85bf7a83f60490b9442'), 'name': 'Sara', 'student_id': '02', 'grade': 95}
Found document:	{'_id': ObjectId('68ecc8c9f7a83f60490b9446'), 'name': 'Sofia', 'student_id': '06', 'grade': 85}


In [38]:
# Update documents

filter = {
    "name": "John",
}

payload = {
    "$set": {"grade": 82}
}

result = collection.update_one(filter, payload, upsert=True)



In [39]:
filter = {
    "grade": {"$gt": 80}
}
matched_documents = collection.find(filter)

for document in matched_documents:
    print(f"Found document:\t{document}")

Found document:	{'_id': ObjectId('68ecc83cf7a83f60490b9441'), 'name': 'Linda', 'student_id': '01', 'grade': 98}
Found document:	{'_id': ObjectId('68ecc85bf7a83f60490b9442'), 'name': 'Sara', 'student_id': '02', 'grade': 95}
Found document:	{'_id': ObjectId('68ecc8c9f7a83f60490b9446'), 'name': 'Sofia', 'student_id': '06', 'grade': 85}
Found document:	{'_id': ObjectId('68ed02714a09ce93040fccc0'), 'name': 'John', 'student_id': '07', 'grade': 82}


In [40]:
# Remove a document

filter = {
    'name': "John"
}

collection.delete_one(filter)

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [42]:
filter = {
    "name": "John"
}
existing_document = collection.find_one(filter)
print(f"Read document:\t{existing_document}")

Read document:	None
